In [2]:
import numpy as np
from PIL import Image
import os
import skimage
from skimage import util
from skimage.util import random_noise
#import cv2

# Assume the linear filter is called a kernel with a default size of 3x3
# filtering can be called cross-correlation. Convolution introduces a 180 shift and is the same as Cross correlation 
# if the convolution matrix is symmetrical
#correlation calculation is the sum of the kernel weights times the pixel values of the image
#in Andrew Ng's convolution slides not stated the kernel is symmetric even though it is so in that example convolution 
#is the same as cross correlation


import scipy.ndimage as ndi
import matplotlib.pyplot as plt

img = ndi.imread('/Users/dc/TestCode/TestMatlab/cameraman.tif')
orig_img = img.copy()

#http://scikit-image.org/docs/stable/api/skimage.util.html#random-noise
#the webposts on creating noise from your own functions are incorrect

#gaussian noise should look blurry
gauss_img = img.copy()
noisy_gauss_image = skimage.util.random_noise(gauss_img, 'gaussian', seed=None, clip=True )

#salt and pepper noise
s_vs_p = 0.5
amount = 0.004

salt_pepper_img = orig_img.copy()
# Salt mode
num_salt = np.ceil(amount * img.size * s_vs_p)
coords = [np.random.randint(0, i - 1, int(num_salt)) for i in img.shape]
salt_pepper_img[coords] = 1

# Pepper mode
num_pepper = np.ceil(amount* img.size * (1. - s_vs_p))
coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in img.shape]
salt_pepper_image[coords] = 0

#speckle
speckle_img = orig_img.copy()
speckle_image = skimage.util.random_noise(speckle_img, 'speckle', seed=None, clip=True)

fig, (ax1,ax2,ax3,ax4) = plt.subplots(ncols=4, sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})

ax1.imshow(orig_img, cmap=plt.cm.gray)
ax1.set_title('Original')
ax1.axis('off')

ax2.imshow(noisy_gauss_image, cmap=plt.cm.gray)
ax2.set_title('Gauss noise')
ax2.axis('off')

ax3.imshow(salt_pepper_image, cmap=plt.cm.gray)
ax3.set_title('salt_pepper')
ax3.axis('off')

ax4.imshow(speckle_image, cmap=plt.cm.gray)
ax4.set_title('speckle')
ax4.axis('off')


plt.show()



In [ ]:
#linear filters can be used to estimate gradients or first derivatives
#http://www.cs.cornell.edu/workshop/curie2015/filtering.pdf

#horizontal partial derivative filter
#these filters have negative weights and use double precision math; not integers

horiz = np.array([[0,0,0],[-1,0,1],[0,0,0]])
horiz_kernel = np.multiply(.5,horiz) 

vert = np.array([[0,-1,0],[0,0,0],[0,1,0]])
vert_kernel = np.multiply(0.5,vert)


#use vert_kernel and horiz_kernel to compute gradient magnitude
grad_mag = np.sqrt(np.add(np.multiply(vert_kernel,vert_kernel), np.multiply(horiz_kernel,horiz_kernel)))

#sobel, roberts, prewitt 1d gradients









In [ ]:
#first gradient approaches
# sobel, prewitt, roberts
from PIL import Image
from skimage.filters import roberts, sobel, scharr, prewitt
from matplotlib import pyplot as plt

img_camera = Image.open('/Users/dc/TestCode/TestMatlab/cameraman.tif')

edge_sobel = sobel(img_camera)
edge_roberts = roberts(img_camera)
edge_scharr = scharr(img_camera)
edge_prewitt = prewitt(img_camera)

fig, (ax0, ax1, ax2, ax3 ) = plt.subplots(ncols=4, sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})

ax0.imshow(edge_roberts, cmap=plt.cm.gray)
ax0.set_title('Roberts Edge Detection')
ax0.axis('off')

ax1.imshow(edge_sobel, cmap=plt.cm.gray)
ax1.set_title('Sobel Edge Detection')
ax1.axis('off')

ax2.imshow(edge_scharr, cmap=plt.cm.gray)
ax2.set_title('Scharr Edge Detection')
ax2.axis('off')

ax3.imshow(edge_prewitt, cmap=plt.cm.gray)
ax3.set_title('Prewitt Edge Detection')
ax3.axis('off')

plt.show()
#plt.tight_layout()

In [ ]:
#laplacian edge detector vs 1d edge detectors
#1st derivative edge detection doesnt behave well under noise;


#filtering for 2d gradient using 0 crossings vs. thresholds. Is this better? Visual demo...











In [ ]:
#canny most resistant to noise compared to other 1d gradient methods
import numpy as np
from scipy import ndimage as ndi
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import matplotlib.pyplot as plt

im = np.zeros((128, 128))
im[32:-32, 32:-32] = 1

im = ndi.rotate(im, 15, mode='constant')
im = ndi.gaussian_filter(im, 4) 
im += 0.2 * np.random.random(im.shape)


# Compute the Canny filter for two values of sigma
edges1 = feature.canny(im)
edges2 = feature.canny(im, sigma=3)

sobel=sobel(im)
prewitt = prewitt(im)
roberts = roberts(im)

fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(8, 3), sharex=True, sharey=True)
fig1, (ax11, ax12, ax13, ax14) = plt.subplots(nrows=1, ncols=4, figsize=(8, 3), sharex=True, sharey=True)

ax1.imshow(im, cmap=plt.cm.jet)
ax1.axis('off')
ax1.set_title('noisy image', fontsize=20)

ax2.imshow(edges1, cmap=plt.cm.gray)
ax2.axis('off')
ax2.set_title('Canny filter, $\sigma=1$', fontsize=20)

ax3.imshow(edges2, cmap=plt.cm.gray)
ax3.axis('off')
ax3.set_title('Canny filter, $\sigma=3$', fontsize=20)

fig.subplots_adjust(wspace=0.02, hspace=0.02, top=0.9,
                    bottom=0.02, left=0.02, right=0.98)

ax11.imshow(im, cmap=plt.cm.jet)
ax11.axis('off')
ax11.set_title('noisy image', fontsize=20)

ax12.imshow(sobel, cmap=plt.cm.gray)
ax12.axis('off')
ax12.set_title('Sobel filter',  fontsize=20)

ax13.imshow(roberts, cmap=plt.cm.gray)
ax13.axis('off')
ax13.set_title('Prewitt filter',  fontsize=20)

ax14.imshow(roberts, cmap=plt.cm.gray)
ax14.axis('off')
ax14.set_title('Roberts filter',  fontsize=20)


fig1.subplots_adjust(wspace=0.02, hspace=0.02, top=0.9,
                    bottom=0.02, left=0.02, right=0.98)

#add modifications for figure inline to python workbook

plt.show()


